In [ ]:
!pip install dali-dataset

     |████████████████████████████████| 1.8MB 4.9MB/s 


In [ ]:
from google.colab import drive
import os
import DALI as dali_code
from math import log2, pow
import pickle as pk
import re
import copy

ModuleNotFoundError: ignored

In [ ]:
drive.mount('/content/drive')

# Confirmation that the drive has been mounted correctly
dali_data_path = '/content/drive/My Drive/Colab Notebooks/DALI'
#os.listdir(drive_path)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Load the dataset, takes time

# Load all the dataset
# dali_data = dali_code.get_the_DALI_dataset(dali_data_path, skip=[], keep=[])

# load sample data
dali_data = dali_code.get_the_DALI_dataset(dali_data_path, skip=[],
                                           keep=['00c4f2efb0b143728113fad1a78d6ce3','00d5c65d644c4a549e7d501d65397b7b','00d9de583ea3498db32ee187f6b66dd6', '00d52567b07b4a819aa6149d2fbfbfd5', '00dae9818fe74c15897e4e7c52e75294'])

# Confirmation of data load
# Should print ['DALI_ID' 'NAME' 'YOUTUBE' 'WORKING'] if working
dali_info = dali_code.get_info(dali_data_path + '/info/DALI_DATA_INFO.gz')
print(dali_info[0])

['DALI_ID' 'NAME' 'YOUTUBE' 'WORKING']


In [ ]:
# Auxilary functions to preprocess dataset
A4 = 440
C0 = A4*pow(2, -4.75)
name = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
def pitch(freq):
    h = round(12*log2(freq/C0))
    octave = h // 12
    n = h % 12
    return name[n] + str(octave)

def round_time_delta(delta):
  d = delta*10
  d = round(d)
  return d/10

In [ ]:
data_song_keys = list(dali_data.keys())
only_char_reg = re.compile("[^a-zA-Z_\']")
ds = []

for song in data_song_keys:
  # get the data
  song_info = dali_data[song].info
  song_annotations = dali_data[song].annotations

  # remove non-english songs
  if song_info['metadata']['language'] != 'english':
    continue

  if song_annotations['type'] != 'vertical':
    dali_data[song].horizontal2vertical()

  # iterate over paragraphs
  for paragraph in song_annotations['annot']['hierarchical']:
    for line in paragraph['text']:
      line_in_notes = []
      line_in_words = ''
      for word in line['text']:
        for note in word['text']:
          note_text = only_char_reg.sub('',note['text'])
          delta = round_time_delta(note['time'][1]-note['time'][0])
          ref_note = [note_text,pitch(note['freq'][0]),delta]
          line_in_notes.append(ref_note)
          line_in_words = line_in_words + note_text
        line_in_words = line_in_words + ' '
      ds.append([line_in_notes,line_in_words])

In [ ]:
output =  '/content/ds.pickle'
with open(output, 'wb') as pickle_file:
    pk.dump(ds,pickle_file,protocol=pk.HIGHEST_PROTOCOL)


In [ ]:
with open(output, 'rb') as pickle_file:
    dss = pk.load(pickle_file)
    #upickler = pk.Unpickler(pickle_file)
    #ds = upickler.load()

In [ ]:
for song in ds:
  print(song[0])